In [1]:
import requests, json
import base64
import datetime
from urllib.parse import urlencode

workflow of the app: 


1) input message
2) search for tracks based on message
3) create playlist based on the tracks received
4) display url of created playlist


Client Id: 

In [12]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expires = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    method = "POST"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("Enter not None client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }
    
    def perform_auth(self):
        token_url = self.token_url
        method = self.method
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        
        r = requests.post(token_url, data=token_data, headers=token_header)
        print(r.json())
        
        if r.status_code not in range(200, 299):
            raise Exception("Auth failed")
        token_response_data = r.json()
        now = datetime.datetime.now()
        self.access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.get_access_token()
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def base_search(self, query_params):
        access_token = self.access_token
        headers = {
            'Authorization': f"Bearer { access_token }"
        }
        endpoint = "https://api.spotify.com/v1/search"
        url = f"{endpoint}?{query_params}"
        r = requests.get(url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    def search(self, query=None, query_type = "track"):
        query_params = urlencode({"q": f"{query}", "type": f"{query_type}"})
        return self.base_search(query_params)

In [13]:
client = SpotifyAPI(client_id, client_secret)

In [14]:
client.perform_auth()
client.search('Time')

{'access_token': 'BQClmrrE3Xk10ZMVB1q7w5wQeuI20fWZqp7pLp5c0UUm5l-_9NcpqY10BUWE7zl2T_XRxXxbIWyQBVWV6hT1AzgPhZNc0TMdiqEjaFwX3-MRcagkIQQ', 'token_type': 'Bearer', 'expires_in': 3600}


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
       'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
       'id': '0TnOYISbd1XYRBk9myaseg',
       'name': 'Pitbull',
       'type': 'artist',
       'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
   

In [36]:
query_params = {
    'client_id': 'b158470a47764f1aa01d8f9c6b4d19ec',
    'response_type': 'code',
    'redirect_uri': 'http://localhost:3000/api/v1/logging-in',
    'scope': "user-library-read",
    'show_dialog': 'true'
}
url = "https://accounts.spotify.com/authorize/"
url = f"{url}?{urlencode(query_params)}"

In [37]:
r = requests.get(url)

In [38]:
r.url

'https://accounts.spotify.com/en/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%2F%3Fclient_id%3Db158470a47764f1aa01d8f9c6b4d19ec%26response_type%3Dcode%26redirect_uri%3Dhttp%253A%252F%252Flocalhost%253A3000%252Fapi%252Fv1%252Flogging-in%26scope%3Duser-library-read%26show_dialog%3Dtrue'

In [39]:
r.json

<bound method Response.json of <Response [200]>>